# How to Prepare TopoFlow Input Files

### Table of Contents

1.  <a href="#intro_B">Introduction</a> <br>
2.  <a href="#choose_B">Set the site_prefix, case_prefix and Common Info</a> <br>
3.  <a href="#dirs_B">Create Some Directories in Your Home Directory</a> <br>
4.  <a href="#define_B">Define Information for a Specific Basin of Interest</a> <br>
5.  <a href="#copy_cfg_B">Copy a Set of CFG files into New Basin Directory</a> <br>
6.  <a href="#path_info_B">Create a New "path_info" CFG File</a> <br>
7.  <a href="#copy_outlets_B">Copy an "outlets file" of Grid Cells to Monitor</a> <br>
8.  <a href="#copy_rain_B">Copy a Simple "rainrates file" into the "met" Directory</a> <br>
9.  <a href="#import_utils_B">Import Some TopoFlow Utilities</a> <br>
10. <a href="#clip_dem_B">Clip a Source DEM to a Bounding Box and Resample</a> <br>
11. <a href="#read_dem_tif_B">Read Clipped DEM from a GeoTIFF File</a> <br>
12. <a href="#read_dem_rtg_B">Read DEM from an RTG File (with RTI File)</a> <br>
13. <a href="#read_dem_nc_B">Read DEM from a netCDF File</a> <br>
14. <a href="#smooth_dem_B">Create a DEM with Smoother Slopes</a> <br>
15. <a href="#fill_pits_B">Fill Depressions in the DEM and Save</a> <br>
16. <a href="#d8_flow_B">Compute a D8 Flow Direction Grid</a> <br>
17. <a href="#d8_area_B">Save the D8 Total Contributing Area (TCA) Grid</a> <br>
18. <a href="#d8_slope_B">Compute the D8 Slope Grid</a> <br>
19. <a href="#d8_aspect_B">Compute the D8 Aspect Grid</a> <br>
20. <a href="#chan_width_B">Compute an Estimated Channel Width Grid</a> <br>
21. <a href="#chan_manning_B">Compute an Estimated "Manning's n" Grid</a> <br>
22. <a href="#chan_sinu_B">Compute an Estimated Channel Sinuosity Grid</a> <br>
23. <a href="#bankfull_d_B">Compute an Estimated Bankfull Depth Grid</a> <br>
24. <a href="#init_depth_B">Compute an Estimated Initial Channel Water Depth Grid</a> <br>
25. <a href="#soil_hydro_B">Create Soil Hydraulic Property Grids (via Pedotransfer) </a> <br>
26. <a href="#rainrates_B">Create a Space-time Rainfall Rates Grid Stack</a> <br>
27. <a href="#setup_B">Appendix 1: Installing TopoFlow in a conda Environment</a>

<!-- Hyperlink IDs must be unique to work in Jupyter Lab when
there are multiple notebooks open. Hence the trailing letter. -->

## Introduction  <a id="intro_B"></a> 

This Jupyter notebook demonstrates how to use a collection of TopoFlow utilities to create required input files for the TopoFlow hydrologic model.  Before you can run the code in this notebook, you will need to install the TopoFlow 3.6 Python package. This notebook uses numerous utilities from the TopoFlow 3.6 package, in <b>topoflow/utils</b>.  Detailed instructions and background information for how to install TopoFlow in a conda environment are given in Appendix 1: Installing TopoFlow in a conda Environment.

This notebook addresses the important issue of <b>portability</b>.  By simply specifying a geographic bounding box and a few other bits of information for <b>any river basin in Ethiopia</b> in the first few cells, you can create all of the input files that are required by TopoFlow (except space-time meteorological grids, like rainfall rates, as discussed near the end.)  With a few other very minor changes, this notebook could create input files required by TopoFlow for <b>any river basin on Earth</b>.  This is because it uses data sets that are available <b>globally</b>, such as the
[<b>MERIT DEM</b>](http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/) and the
[<b>ISRIC SoilGrids</b>](https://www.isric.org/explore/soilgrids) database
[<b>and services</b>](https://soilgrids.org/).

This notebook currently assumes that the user has access to a shared Dropbox folder for the World Modelers MINT Project called <b>MINT</b>, and it copies some necessary files from there at the beginning.  However, these can be provided in a zip file upon request. (Scott.Peckham@colorado.edu).

This notebook is for TopoFlow pre-processing (preparation of input files).  There are several other notebooks, including TopoFlow_Getting_Started.ipynb and TopoFlow_Visualization_v2.ipynb, which address other issues.

<b>Note</b> Once you have set up some choices in the first few cells, it is possible to choose
Run > Run All Cells.

## Set the site_prefix, case_prefix and Common Info  <a id="choose_B"></a> 

In the next cell, uncomment the site_prefix for the Ethiopian river basin that you want to create input files for.  This notebook has been pre-configured with 6 Ethiopian river basins, but if you add a cell here at the top, similar to the ones for the other 6 basins, you can use this notebook to create all of the TopoFlow input files for <b>any</b> river basin in Ethiopia.

In [1]:
site_prefix = 'Baro-Gam_60sec'
# site_prefix = 'Awash-border_60sec'
# site_prefix = 'Shebelle-Imi_60sec'
# site_prefix = 'Ganale-Welmel_60sec'
# site_prefix = 'Guder_30sec'
# site_prefix = 'Muger_30sec'

#--------------------------------------------------------
# For now, keep these the same for all the river basins
#--------------------------------------------------------
case_prefix  = 'Test1'
channel_width_power = 0.5
max_sinuosity   = 1.3    # [m/m]
min_manning_n   = 0.03   # [m / s^(1/3)]
max_manning_n   = 0.2    # [m / s^(1/3)]
bankfull_depth_power = 0.4
bank_angle = 30.0   # [degrees]

## Create Some Directories in Your Home Directory  <a id="dirs_B"></a>

This notebook uses some data sets that provide coverage for the entire country of Ethiopia.  These have been placed in a Dropbox folder that is shared by the
[<b>MINT Project</b>](http://mint-project.info/).  If you don't have access to the MINT Dropbox folder, you can request the files as a zip file, copy them somewhere on your computer and edit <b>src_dir</b> in the next cell accordingly.

In [2]:
import glob
import os, os.path
import shutil

home_dir     = os.path.expanduser("~")
src_dir      = home_dir + '/Dropbox/MINT/Data/DEMs/Ethiopia/'
src_soil_dir = src_dir + 'isric_soil_data/'
src_cfg_dir  = src_dir + 'test1_cfg_files/'

test_dir  = home_dir + '/TF_Tests3/'     #############################
basin_dir = test_dir + site_prefix + '/'
topo_dir  = basin_dir + '__topo/'
soil_dir  = basin_dir + '__soil/'
met_dir   = basin_dir + '__met/'
cfg_dir   = basin_dir + 'Test1_cfg/'

if not(os.path.exists( test_dir )):  os.mkdir( test_dir )
if not(os.path.exists( basin_dir )): os.mkdir( basin_dir )
if not(os.path.exists( topo_dir )):  os.mkdir( topo_dir )
if not(os.path.exists( soil_dir )):  os.mkdir( soil_dir )
if not(os.path.exists( met_dir )):   os.mkdir( met_dir )
if not(os.path.exists( cfg_dir )):   os.mkdir( cfg_dir )

print( 'Home directory  =', home_dir )
print( 'Basin directory =', basin_dir )
print( 'CFG directory   =', cfg_dir )
print()


Home directory  = /Users/peckhams
Basin directory = /Users/peckhams/TF_Tests3/Baro-Gam_60sec/
CFG directory   = /Users/peckhams/TF_Tests3/Baro-Gam_60sec/Test1_cfg/



## Define Information for a Specific Basin of Interest  <a id="define_B"></a> 

TopoFlow uses a <b>site_prefix</b> for all of the filenames in a data set that pertain to the geographic location (the "site").  These files describe static properties of the location, such as topography and soil.  The default site prefix in this notebook is <b>Baro-Gam_60sec</b>.

Topoflow uses a <b>case_prefix</b> for all of the filenames in a data set that describe a particular model scenario (the "case" under consideration).  These files describe things that can change from one model run to the next for the same site.  The default case prefix in this notebook is <b>Test1</b>.  Note that component CFG filenames always start with the case prefix.

By simply changing the information in this next code block, this notebook can generate TopoFlow input files for any river basin in Ethiopia.  However, please heed the <b>Important Warning</b> below.

out_bounds = Geographic bounding box for the chosen river basin<br>
out_xres_sec = the spatial grid cell xsize to use, in arcseconds<br>
out_yres_sec = the spatial grid cell ysize to use, in arcseconds<br>

### Important Warning !!

Some of the input grid files generated by this notebook are computed using empirical, power-law estimates, based on a grid of <b>total contributing area</b> (TCA).

For these, it is necessary -- <b>at a minimum</b>  -- to know the values of a few key variables at the basin outlet.  These must be determined from the literature or other data sets.  These variables that are specific to a particular river basin have been collected <b>in the next code block</b>.

If these are not set to reasonable values, the resulting predictions will be meaningless.

### Use info for Baro River at Gambella ?

In [3]:
if (site_prefix == 'Baro-Gam_60sec'):

    out_xres_sec = 60.0    # [arcseconds]
    out_yres_sec = 60.0    # [arcseconds]
    # Set the geographic bounding box and the grid cell size that
    # will be used for the TopoFlow model run, where
    #    Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    # Next one gives ncols = 133, nrows = 128.
    ## out_bounds = [ 34.22125, 7.3704166666, 36.43791666666, 9.50375]
    # Next one gives ncols = 134, nrows = 129
    out_bounds = [ 34.221249999999, 7.362083333332, 36.450416666666, 9.503749999999]

    max_river_width = 140.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 23567.7  # total contributing area at basin outlet [km2]
    Qbase_out = 40.0     # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 8.0   # [meters]    # from literature or data


### Use info for Awash River at Oromia border ?

In [4]:
if (site_prefix == 'Awash-border_60sec'):

    out_xres_sec = 60.0    # [arcseconds]
    out_yres_sec = 60.0    # [arcseconds]
    # Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    # out_bounds = [  37.829583333333, 6.657916666666, 39.929583333333,  9.374583333333]  # (for 30 arcseconds)
    # out_bounds = [37.829583333333, 6.657916666666, 39.929583333333,  9.374583333333]  # (for 60 arcseconds)
    out_bounds = [37.829583333333, 6.654583333333, 39.934583333333, 9.374583333333]
    max_river_width = 25.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 30679.5   # total contributing area at basin outlet [km2]
    Qbase_out = 10.0      # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 4.0   # [meters]    # from literature or data


### Use info for Shebelle River at Imi ?

In [5]:
if (site_prefix == 'Shebelle-Imi_60sec'):
    
    out_xres_sec = 60.0    # [arcseconds]
    out_yres_sec = 60.0    # [arcseconds]
    # Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    # out_bounds = [ 38.159583333333, 6.324583333333, 43.559583333333, 9.899583333333]  # (for 30 arcseconds)
    out_bounds = [38.159583333333, 6.319583333333, 43.559583333333, 9.899583333333] 
    max_river_width = 130.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 90662.1   # total contributing area at basin outlet [km2]
    Qbase_out = 50.0      # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 7.0   # [meters]    # from literature or data


### Use info for Ganale River at border ?

In [6]:
# Actually, this tributary of the Ganale River is called the Welmel Shet River.
# The outlet is at the border between Oromia and Somali Regions.

if (site_prefix == 'Ganale-Welmel_60sec'):
    
    out_xres_sec = 60.0    # [arcseconds]
    out_yres_sec = 60.0    # [arcseconds]
    # Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    out_bounds = [39.174583333333, 5.527916666666, 41.124583333333, 7.098749999999]  # (3 arcseconds) 
    max_river_width = 40.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 15241.7   # total contributing area at basin outlet [km2]
    Qbase_out = 3.0      # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 2.0   # [meters]    # from literature or data    
    

### Use info for Guder River at Blue Nile confluence ?

In [7]:
if (site_prefix == 'Guder_30sec'):
    out_xres_sec = 30.0    # [arcseconds]
    out_yres_sec = 30.0    # [arcseconds]
    # Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    out_bounds = [37.149583333333, 8.596250000000, 38.266250000000, 9.904583333333]
    max_river_width = 20.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 6487.8   # total contributing area at basin outlet [km2]
    Qbase_out = 2.0      # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 2.0   # [meters]    # from literature or data


### Use info for Muger River at Blue Nile confluence ?

In [8]:
if (site_prefix == 'Muger_30sec'):
    out_xres_sec = 30.0    # [arcseconds]
    out_yres_sec = 30.0    # [arcseconds]
    # Bounds = [ minlon, minlat, maxlon, maxlat ]
    # The bounding box MUST contain the entire watershed polygon.
    out_bounds = [37.807916666667, 8.929583333333, 39.032916666667, 10.112916666667]
    max_river_width = 45.0  # [meters]  from Google Maps or Google Earth
    A_out_km2 = 6924.12    # total contributing area at basin outlet [km2]
    Qbase_out = 3.0      # estimated baseflow discharge at basin outlet [m^3 / s]
    max_bankfull_depth = 2.0   # [meters]    # from literature or data

## Copy a DEM for Ethiopia into the New Basin Directory  <a id="copy_dem_B"></a>

A DEM for the entire country of Ethiopia, saved in GeoTIFF format and with a grid cell size of 3 arcseconds can be found in a shared Dropbox folder called MINT.  Copy this file into the <b>topo directory</b> you just created.


In [9]:
# os.chdir( topo_dir )

# dem_file = 'Ethiopia_MERIT_DEM.tif'
# src_file = src_dir + dem_file
# dst_file = dem_file
# shutil.copyfile( src_file, dst_file )

## Copy a Set of CFG files into New Basin Directory  <a id="copy_cfg_B"></a>

A set of default CFG files to configure TopoFlow can be also be found in the MINT Dropbox folder.  Copy this file into the <b>cfg1 directory</b> you just created.

In [10]:
os.chdir( src_cfg_dir )
cfg_file_list = sorted( glob.glob( '*.cfg' ) )

os.chdir( cfg_dir )
for cfg_file in cfg_file_list:
    shutil.copyfile( src_cfg_dir + cfg_file, cfg_file)

# Copy the default "provider_file"
provider_file = case_prefix + '_providers.txt'
shutil.copyfile( src_cfg_dir + provider_file, provider_file )

'Test1_providers.txt'

## Create a New "path_info" CFG File  <a id="path_info_B"></a>

In [11]:
os.chdir( cfg_dir )
cfg_file = case_prefix + '_path_info.cfg'
cfg_unit = open( cfg_file, 'w')

d1 = basin_dir
d2 = d1
cp = case_prefix
sp = site_prefix
bar = '#===============================================================================\n'
cfg_unit.write( bar )
cfg_unit.write('# TopoFlow Config File for: Path_Information\n')
cfg_unit.write( bar )
L1 = 'in_directory  | XX | string | input directory'.replace('XX', d1)
L2 = 'out_directory | XX | string | output directory'.replace('XX', d2)
L3 = 'site_prefix   | XX | string | file prefix for the study site'.replace('XX', sp)
L4 = 'case_prefix   | XX | string | file prefix for the model scenario'.replace('XX',cp)
cfg_unit.write( L1 + '\n')
cfg_unit.write( L2 + '\n')
cfg_unit.write( L3 + '\n')
cfg_unit.write( L4 + '\n')
cfg_unit.close()

## Copy an "outlets file" of Grid Cells to Monitor   <a id="copy_outlets_B"></a>

In TopoFlow component CFG files, flags can be set to tell TopoFlow to write values of chosen gridded variables to a file, to create a <b>grid stack</b>, indexed by time.  Grids are saved at a time interval set by <b>save_grid_dt</b>.

Other flags in a CFG file can be set to tell TopoFlow to write values of chosen gridded variables to a file, but only at a specified set of grid cells.  These "monitored grid cells" or "virtual gauges" are set in an <b> outlets file</b> named <b>[case_prefix]_outlets.txt</b>.  It is a simple, multi-column text file, like this:
```
------------------------------------------------------------
 Monitored Grid Cell (Outlet) Information
-------------------------------------------------------------------------------------
    Column       Row     Area [km^2]      Relief [m]      Lon [deg]     Lat [deg]
-------------------------------------------------------------------------------------
        4         79        24647.3         2433.62       34.296250     8.1787500
       14         76        24011.0         2425.90       34.462917     8.2287500
       29         79        22972.0         2393.16       34.712917     8.1787500
       48         59          507.3         1434.04       35.029583     8.5120833
```
It is not necessary for the Area and Relief columns to contain valid values (they are for reference, but are unused).  However, the column, row, longitude and latitude of each grid cell to be monitored must be specified.  They must match columns and rows in the DEM that is being used for the model run.  Creating an outlets file therefore requires a <b>human in the loop</b>, using interactive GIS (Geographic Information System) software and making intelligent choices.  It cannot be automated.

Here we just copy an existing outlets file for one of the 6 Ethiopian basins.

In [12]:
os.chdir( cfg_dir )

outlets_dir = src_dir + 'outlet_files_v2/' + site_prefix + '/'
outlet_file = case_prefix + '_outlets.txt'
shutil.copyfile( outlets_dir + outlet_file, outlet_file)

'Test1_outlets.txt'

## Copy a Simple "rainrates file" into the "met" Directory   <a id="copy_rain_B"></a>

In this step we copy a file with a simple time series of spatially uniform rainrates for testing purposes.  See the cell near the end about preparing space-time rainfall grid stacks.


In [13]:
## os.chdir( met_dir )
rain_file = 'Test1_rain_rates.txt'
shutil.copyfile( src_cfg_dir + rain_file, met_dir + rain_file)

'/Users/peckhams/TF_Tests4/Baro-Gam_60sec/__met/Test1_rain_rates.txt'

## Import Some TopoFlow Utilities  <a id="import_utils_B"></a>

In [4]:
from topoflow.utils import regrid
from topoflow.utils import import_grid
from topoflow.utils import fill_pits
from topoflow.utils import rtg_files
from topoflow.utils import rti_files
from topoflow.utils import parameterize
from topoflow.utils import init_depth
from topoflow.utils import pedotransfer

from topoflow.components import d8_global
from topoflow.components import smooth_DEM

Importing TopoFlow 3.6 package modules:
   topoflow.utils
   topoflow.utils.tests
   topoflow.components
   topoflow.components.tests
   topoflow.framework
   topoflow.framework.tests
 


## Clip a Source DEM to a Bounding Box and Resample  <a id="clip_dem_B"></a>

Here, we use the TopoFlow <b>regrid</b> utility to <b>clip</b> a DEM for the entire country of Ethiopia to the geographic bounding box for a particular river basin in Ethiopia.  This utility uses the gdal.warp() function in the GDAL Python package.  For this example, we use the Baro River basin which lies mostly in the Oromia region but drains past the town of Gambella into the Gambella region.

At the same time, we <b>resample</b> (via spatial bilinear interpolation) the resulting DEM to a different, coarser spatial resolution.  The source DEM has a grid cell size of 3 arcseconds (roughly 90 meters), while the new DEM has a grid cell size of 60 arcseconds (roughly 1800 meters).  Both the source DEM and new DEM are stored in GeoTIFF format.  Resampling typically causes the bounding box to change slightly.

In [15]:
### in_nodata = ????
### out_nodata = -9999.0

os.chdir( topo_dir )
in_file  = src_dir + 'Ethiopia_MERIT_DEM.tif'
out_file = site_prefix + '_rawDEM.tif'

regrid.regrid_geotiff(in_file=in_file, out_file=out_file, 
                   out_bounds=out_bounds,
                   out_xres_sec=out_xres_sec, out_yres_sec=out_yres_sec,
                   ### in_nodata=None, out_nodata=None, 
                   RESAMPLE_ALGO='bilinear', REPORT=True)

Input grid file:
   /Users/peckhams/Dropbox/MINT/Data/DEMs/Ethiopia/Ethiopia_MERIT_DEM.tif
   ncols  = 20401
   nrows  = 16801
   xres   = 3.0  [arcsecs]
   yres   = 3.0  [arcsecs]
   bounds = [31.999583333333, 1.999583333333666, 49.00041666666634, 16.000416666667]
   gmin   = -197.44768
   gmax   = 4514.622

Output grid file:
   Baro-Gam_60sec_rawDEM.tif
   ncols  = 134
   nrows  = 129
   xres   = 60.0  [arcsecs]
   yres   = 60.0  [arcsecs]
   bounds = [34.221249999999, 7.353749999999, 36.45458333333234, 9.503749999999]
   gmin   = 422.53043
   gmax   = 2854.459
Finished regridding.



## Read Clipped DEM from a GeoTIFF File  <a id="read_dem_tif_B"></a>

Here we import a DEM in GeoTIFF format.  Note that NetCDF (.nc) and RiverTools Grid (RTG) formats can also be imported.

Most of the TopoFlow utilities use grids saved in the RiverTools Grid (RTG) format, which is a generic, binary, row-major format.  Georeferencing information for the grid is stored in a small, separate text file in RiverTools Info (RTI) format.  When the rti_file argument is specified, georeferencing information is also saved in the RTI file format for later use.

In [14]:
os.chdir( topo_dir )

tif_file = site_prefix + '_rawDEM.tif'
rti_file = site_prefix + '.rti'
DEM = import_grid.read_from_geotiff( tif_file, REPORT=True,
                                     rti_file=rti_file)

grid_info = rti_files.read_info( rti_file )

ERROR:  Wrong filename or file does not exist.
ERROR: Could not find RTI file for:
      Baro-Gam_60sec.rti
 


## Read DEM from an RTG File (with RTI file)  <a id="read_dem_rtg_B"></a>

<b>Note:</b> This shows how to alternately import a DEM in RTG format, but is commented out for now.

In [17]:
# os.chdir( topo_dir )
# rtg_file = site_prefix + '_rawDEM.rtg'
# DEM = import_grid.read_from_rtg( rtg_file, REPORT=True)
# grid_info = rti_files.read_info( rtg_file )

## Read DEM from a netCDF File  <a id="read_dem_nc_B"></a>

<b>Note:</b> This shows how to alternately import a DEM in netCDF format, but is commented out for now.

In [18]:
# os.chdir( topo_dir )
# nc_file = site_prefix + '_rawDEM.nc'
# DEM = import_grid.read_from_netcdf( nc_file, REPORT=True)

## Create a DEM with Smoother Slopes  <a id="smooth_dem_B"></a>

<b>Note:</b> This step is optional and has been commented out for now.

For so-called <b>mature</b> landscapes this "profile smoothing" algorithm works well, and results in a DEM with smoothly decreasing, nonzero channel slopes everywhere.  However, the landscape of the Baro River basin is not a good candidate because it is not mature.

In [19]:
# os.chdir( topo_dir )
# c = smooth_DEM.DEM_smoother()
# c.DEBUG = True
# case_prefix = 'Test1'
# cfg_file = case_prefix + '_dem_smoother.cfg'
# c.initialize( cfg_file=cfg_file, mode='driver')
# c.update()

## Fill Depressions in the DEM and Save <a id="fill_pits_B"></a>

This step is necessary to make the DEM <b>hydrologically sound</b>.

In [20]:
# data_type = 'FLOAT'
# shp = DEM.shape
# nrows = shp[0]
# ncols = shp[1]

os.chdir( topo_dir )
data_type = grid_info.data_type   # e.g. "INTEGER" or "FLOAT"
ncols = grid_info.ncols
nrows = grid_info.nrows

fill_pits.fill_pits( DEM, data_type, ncols, nrows, SILENT=False)

new_DEM_file = site_prefix + '_DEM.rtg'
rtg_files.write_grid( DEM, new_DEM_file, grid_info, SILENT=False)

Number of nodata and NaN values = 0
Finished initializing "closed" array.
 
Putting boundary pixels on heap...
Number of pixels on heap = 522
Finished with heap insertion.
 
n_closed = 0 of 17286
n_closed = 5000 of 17286
n_closed = 10000 of 17286
n_closed = 15000 of 17286
Total pixels   = 17286
Raised  pixels = 783
Drained pixels = 17288
Run time for fill_pits() =     0.5058 [seconds]
Finished with fill_pits().
 
Writing grid values...
Finished writing grid to:
    Baro-Gam_60sec_DEM.rtg


## Compute the D8 Flow Direction Grid  <a id="d8_flow_B"></a>

TopoFlow includes a component called <b>d8_global</b> that can compute a grid of D8 flow direction codes (Jenson 1984 convention), as well as several additional, related grids such as a grid of total contributing area (TCA).  TopoFlow components are configured through the use of configuration files, which are text files with the extension ".cfg".  Therefore, we now need to copy 2 CFG files into our working directory, called:  <i>Test1_path_info.cfg</i> (for path information) and <i>Test1_d8_global.cfg</i>.

In [21]:
os.chdir( topo_dir )
d8 = d8_global.d8_component()
d8.DEBUG = False
cfg_file = cfg_dir + case_prefix + '_d8_global.cfg'  # (need full path here)
time = 0.0
d8.initialize( cfg_file=cfg_file, SILENT=False, REPORT=True )
d8.update( time, SILENT=False, REPORT=True )

rti_file = site_prefix + '.rti'
grid_info = rti_files.read_info( rti_file, REPORT=True )

d8_code_file = site_prefix + '_flow.rtg'
rtg_files.write_grid(d8.d8_grid, d8_code_file, grid_info, RTG_type='BYTE')

D8 component: Initializing...
Computing pixel area grid...
Computing pixel dimensions...
    min(dx), max(dx) = 1830.1156, 1840.1654 [m]
    min(dy), max(dy) = 1843.2078, 1843.4075 [m]
    min(dd), max(dd) = 2597.59 , 2604.539 [m]
    min(da), max(da) = 3373648.7317201453, 3391807.13024007 [m^2]
Computing pixel IDs...
Computing edge pixel IDs...
Computing "not_edge" grid...
Computing "resolve array"...
Reading grid values...
Finished reading grid from:
  /Users/peckhams/TF_Tests4/Baro-Gam_60sec/__topo/Baro-Gam_60sec_DEM.rtg
   min(DEM), max(DEM) = 422.70148 2854.459
 
Filling pits in initial DEM...
Number of nodata and NaN values = 0
Finished initializing "closed" array.
 
Putting boundary pixels on heap...
Number of pixels on heap = 522
Finished with heap insertion.
 
n_closed = 0 of 17286
n_closed = 5000 of 17286
n_closed = 10000 of 17286
n_closed = 15000 of 17286
Total pixels   = 17286
Raised  pixels = 0
Drained pixels = 17288
Run time for fill_pits() =     0.4651 [seconds]
Finished

## Save the D8 Total Contributing Area (TCA) Grid  <a id="d8_area_B"></a>

In [22]:
os.chdir( topo_dir )
d8_area_file = site_prefix + '_d8-area.rtg'
rtg_files.write_grid( d8.A, d8_area_file, grid_info, RTG_type='FLOAT', SILENT=False)

Writing grid values...
Finished writing grid to:
    Baro-Gam_60sec_d8-area.rtg


## Compute the D8 Slope Grid  <a id="d8_slope_B"></a>

In [23]:
os.chdir( topo_dir )
d8.update_slope_grid()
d8_slope_file = site_prefix + '_slope.rtg'
rtg_files.write_grid( d8.S, d8_slope_file, grid_info, RTG_type='FLOAT', SILENT=False)

Writing grid values...
Finished writing grid to:
    Baro-Gam_60sec_slope.rtg


## Compute the D8 Aspect Grid  <a id="d8_aspect_B"></a>

In [24]:
os.chdir( topo_dir )
d8.update_aspect_grid()
d8_aspect_file = site_prefix + '_aspect.rtg'
rtg_files.write_grid(d8.aspect, d8_aspect_file, grid_info, RTG_type='FLOAT', SILENT=False)

Writing grid values...
Finished writing grid to:
    Baro-Gam_60sec_aspect.rtg


## Compute the Estimated Channel Width Grid <a id="chan_width_B"></a>

First, use Google Maps or Google Earth to estimate the width of the river at the outlet to your river basin, in meters.  Here, we'll assume that width equals 140 meters.

The idea is to estimate the channel widths throughout the basin (as a grid with the same dimensions as the DEM), using an empirical power law of the form:  $w = c \, A^p$
where A is the total contributing area (TCA) that we computed as a grid above and saved into "d8_area_file".  A typical value of p is 0.5.  The value that w should have where A is maximum (e.g. the river outlet) is specified as g1. 

In [25]:
# Should be specified at the top of this notebook.
# max_river_width = 140.0  # [meters]
# channel_width_power = 0.5

os.chdir( topo_dir )
width_file = site_prefix + '_chan-w.rtg'
parameterize.get_grid_from_TCA(site_prefix=site_prefix, topo_dir=topo_dir,
             area_file=d8_area_file, out_file=width_file,
             g1=max_river_width, p=channel_width_power)

Power-law parameters are:
c = 0.8907117944769954
p = 0.5
Values set to 1.0 where A <= 0.
  This occurred at 576 grid cells.
grid min = 1.0
grid_max = 139.99998
Finished writing file: 
Baro-Gam_60sec_chan-w.rtg



## Compute the Estimated "Manning's n" Grid  <a id="chan_manning_B"></a>

In order to compute grids of river flow velocity and discharge (volume flow rate), a very well-known, empirical formula known as <b>Manning's formula</b> (see Wikipedia) is the method used by default within TopoFlow.  This formula includes a parameter called <b>Manning's n</b>, that characterizes the roughness of the channel bed and resulting frictional loss of momentum.  Typical values in larger river channels range between 0.03 and 0.05.  Manning's formula can also be used for non-channelized, overland flow, but then a much larger value of 0.2 to 0.3 should be used.

The following code uses a power-law estimate of the form:  $n = c \, A^p$, where A is the total contributing area (TCA) grid, to create a grid of Manning's n values.  The value that n should have where A is maximum (e.g. the river outlet) is set as <b>g1</b>.  Similarly, the value that n should have where A is minimum (e.g. on a ridge) is set as <b>g2</b>.  The coefficient, c, and power, p, are then set to match these constraints.

In [26]:
# Should be specified at the top of this notebook.
# min_manning_n = 0.03
# max_manning_n = 0.2

os.chdir( topo_dir )
manning_file = site_prefix + '_chan-n.rtg'
parameterize.get_grid_from_TCA(site_prefix=site_prefix, topo_dir=topo_dir,
             area_file=d8_area_file, out_file=manning_file,
             g1=min_manning_n, g2=max_manning_n )

Power-law parameters are:
c = 0.2591911840780398
p = -0.21319042064432686
Values set to 1.0 where A <= 0.
  This occurred at 576 grid cells.
grid min = 0.030000001
grid_max = 1.0
Finished writing file: 
Baro-Gam_60sec_chan-n.rtg



## Compute the Estimated Channel Sinuosity Grid  <a id="chan_sinu_B"></a>

There are different definitions of channel sinuosity.  Here we are referring to the <b>absolute sinuosity</b>, defined as the ratio of the <b><i>along-channel flow distance</i></b> between the two endpoints of a channel and the <b><i>straight-line distance</i></b> between those endpoints.

By this definition, sinuosity is <b>dimensionless</b> \[km/km\], with a minimum possible value of 1.0.  It tends to increase slowly from 1 where TCA is small to a larger value where TCA is big, but typically does not exceed 1.3.

The following code uses a power-law estimate of the form:  $s = c \, A^p$, where A is the total contributing area (TCA) grid, to create a grid of sinuosity values.  The value that s should have where A is maximum (e.g. the river outlet) is set as <b>g1</b>.  Similarly, the value that n should have where A is minimum (e.g. near a ridge) is set as <b>g2</b>.  The coefficient, c, and power, p, are then set to match these constraints.

In [27]:
# Should be specified at the top of this notebook.
# max_sinuosity = 1.3

os.chdir( topo_dir )
min_sinuosity = 1.0  # (BY DEFINITION.  DO NOT CHANGE.)

sinu_file = site_prefix + '_sinu.rtg'
parameterize.get_grid_from_TCA(site_prefix=site_prefix, topo_dir=topo_dir,
             area_file=d8_area_file, out_file=sinu_file,
             g1=max_sinuosity, g2=min_sinuosity )

Power-law parameters are:
c = 0.9647820441995061
p = 0.029483400285421825
Values set to 1.0 where A <= 0.
  This occurred at 576 grid cells.
grid min = 1.0
grid_max = 1.3
Finished writing file: 
Baro-Gam_60sec_sinu.rtg



## Compute the Estimated Bankfull Depth Grid  <a id="bankfull_d_B"></a>

The <b>bankfull depth</b> is the maximum in-channel water depth of a river at a given location.  (It varies throughout a river basin.)  When the depth of water in a river exceeds this depth, <b>overbank flow</b> occurs and water enters the flood plain adjacent to the channel. <b>Overbank flow depth</b>, <b>inundation depth</b> or simply <b>flooding depth</b> are terms that refer to the depth of water on land outside of the river channel.  It is important to know the bankfull depth in order to more accurately predict the flooding depth. 

While remote sensing images can be used to estimate a river's bankfull width, the river bed typically cannot be "seen" through the water.  Moreover, bankfull depth is typically only measured at a few locations (e.g. at gauging stations) within a river basin, so accurate values of bankfull depth are difficult to obtain.

The following code uses a power-law estimate of the form:  $d_b = c \, A^p$, where A is the total contributing area (TCA) grid, to create a grid of bankfull depth values.  The value that $d_b$ should have where A is maximum (e.g. the river outlet) is set as <b>g1</b>.  A typical, empirical value for p is 0.4. The coefficient, c, is then set to match these constraints.

In [28]:
# Should be specified at the top of this notebook.
# max_bankfull_depth = 8.0  #### This must be determined from literature or data.
# bankfull_depth_power = 0.4

os.chdir( topo_dir )
dbank_file = site_prefix + '_d-bank.rtg'
parameterize.get_grid_from_TCA(site_prefix=site_prefix, topo_dir=topo_dir,
        area_file=d8_area_file,  out_file=dbank_file,
        g1=max_bankfull_depth, p=bankfull_depth_power )

Power-law parameters are:
c = 0.1399514237399525
p = 0.4
Values set to 1.0 where A <= 0.
  This occurred at 576 grid cells.
grid min = 0.2276279
grid_max = 8.0
Finished writing file: 
Baro-Gam_60sec_d-bank.rtg



## Compute the Estimated Initial Channel Water Depth Grid  <a id="init_depth_B"></a>

Here we attempt to estimate the initial depth of water for every channel in the river network.  This is supposed to be the "normal depth" of the river that is maintained by baseflow from groundwater (i.e. due to the groundwater table intersecting the channel bed) and is not attributed to a recent rainfall event.  This is the starting or initial condition for a model run.

This routine uses a <b>grid-based Newton-Raphson</b> iterative scheme to solve a transcendental equation (see Wikipedia) for the initial depth of water in a channel network that results from groundwater baseflow.  The variables involved are:

w = bed bottom width, trapezoid [m]<br>
A = upstream area [$km^2$]<br>
S = downstream slope [m/m]<br>
n = Manning roughness parameter  [$s/m^{1/3}$]<br>
$\theta$ = bank angle [degrees]<br>
d = water depth in channel [m]<br>
$A_c$ = wetted cross-section area [$m^2$]<br>
P  = wetted cross-section perimeter [m]<br>
$R_h = (A_c / P)$ = hydraulic radius [m]<br>
B = spatially-uniform baseflow volume flux [$m s^{-1}$]<br>

The equations used here are: <br>
$Q = v \, A_c = B \,A$    [$m^3 s^{-1}$] (steady-state) <br>
$v = (1/n) \, {R_h}^{2/3} \, S^{1/2} \,\,\,$  [SI units] <br>
$R_h = A_c / P$ <br>
$A_c = d \, [w + (d \, \tan(\theta))]$ <br>
$P = w + [2 \, d \, / \cos(\theta)]$ <br>

Note that B can be estimated from a baseflow discharge measured at the basin outlet.

If we are given w, n, theta, A, S and B, then we get an equation for d that cannot be solved in closed form.  However, we can write the equation $v \, A_c = B \, A$ in the form needed to solve for d (in every grid cell) by Newton's method, i.e.:
$F(d) = [v(d) \, A_c(d)] - (B \, A) = 0$.

In [29]:
# Should be specified at the top of this notebook.
# A_out_km2 = 23567.7  # TCA at basin outlet of Baro River (at Gambella)  [km2]
# Qbase_out= 40.0      # estimated baseflow discharge at basin outlet [m^3 / s]

os.chdir( topo_dir )
B_mps = init_depth.get_baseflow_volume_flux( A_out_km2, Qbase_out, REPORT=True)

d0_file = site_prefix + '_d0.rtg'
init_depth.compute_initial_depth( site_prefix=site_prefix, cfg_dir=cfg_dir,
           SILENT=False, baseflow_rate=B_mps, bank_angle=bank_angle,
           # angle_file=angle_file,
           area_file=d8_area_file, slope_file=d8_slope_file,
           width_file=width_file, manning_file=manning_file,
           sinu_file=sinu_file, d0_file=d0_file)

Baseflow volume flux = 1.697238169189187e-09 [m s-1]
Baseflow volume flux = 0.006110057409081073 [mm h-1]
size(slope) = 17286
size(wb) = 1305
-------------------------------------------------
         Replacing them with smallest slope.
         Use "Profile smoothing tool" instead.
         min(slope) = 4.7353055e-06
         max(slope) = 0.3151615
-------------------------------------------------
 
Iterating...
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17286
Pixels left = 17252
Pixels left = 17165
Pixels left = 17054
Pixels left = 16850
Pixels left = 16550
Pixels left = 16175
Pixels left = 15532
Pixels left = 14581
Pixels left = 13149
Pixels left = 10123
Pixels left = 2940
Pixels left = 576
Pixels left = 576
Pixels left = 0
Finished writing file: 
Baro-Gam_60sec_d0.rtg
d_min = 0.0006527226759138755 [m]
d_max = 2.53

## Create Soil Hydraulic Property Grids (via Pedotransfer) <a id="soil_hydro_B"></a>

Here, we first read a set of ISRIC <b>soil property</b> grids for each of 7 soil layers (see above) in GeoTIFF format.  We then <b>clip</b> them to a chosen geographic bounding box and <b>resample</b> (or regrid) them to a new spatial resolution (grid cell size).  (Typically, we regrid to the TopoFlow model grid.)  Finally, we use
[<b>pedotransfer functions</b>](https://en.wikipedia.org/wiki/Pedotransfer_function)
(Wosten et al., 1998, 2001 ) to compute a corresponding set of <b>soil hydraulic property</b> grids that are used to compute infiltration. (This uses <b>pedotransfer.py</b> in topoflow/utils.  These grids are referenced in the CFG files for the TopoFlow infiltration components.  The REPORT flag can be set to True to see more detailed information about each of the soil property grids.  Any warnings or errors are printed regardless.  In some cases, out-of-range values are generated and by default are forced into the valid range (from above or below).  Spurious values can cause the infiltration model to become [<b>numerically unstable</b>](https://en.wikipedia.org/wiki/Numerical_stability).

Files in the <b>src_soil_dir</b> folder span the entire country of Ethiopia and were downloaded from the
[<b>ISRIC SoilGrids website</b>](https://soilgrids.org/).  ISRIC SoilGrids provides <b>global</b> coverage at two different spatial resolutions:  250m and 1km.  (The products actually use Geographic coordinates and resolutions are 7.5 and 30 arcseconds.  The north-south dimension of grid cells therefore varies with latitude, being roughly 250m or 1km at the equator, but smaller at other latitudes.)
They contain soil property variables for each of 7 soil layers, as follows.

```
Variables:
BLDFIE = Bulk density [kg / m3]
BDRICM = Absolute depth to bedrock [cm]
CLYPPT = Mass fraction of clay [%]
ORCDRC = Soil organic carbon content (fine earth fraction)  [g / kg]
SLTPPT = Mass fraction of silt [%]
SNDPPT = Mass fraction of sand [%]

Layer 1 = sl1 = 0.00 to 0.05 m   (0  to  5 cm)
Layer 2 = sl2 = 0.05 to 0.15 m   (5  to 15 cm)
Layer 3 = sl3 = 0.15 to 0.30 m   (15 to 30 cm)
Layer 4 = sl4 = 0.30 to 0.60 m   (30 to 60 cm)
Layer 5 = sl5 = 0.60 to 1.00 m   (60 to 100 cm)
Layer 6 = sl6 = 1.00 to 2.00 m   (100 to 200 cm)
Layer 7 = sl7 = 2.00 to ??   m   (200 to ??? cm)
```

#### ISRIC Data Citation

Hengl T, Mendes de Jesus J, Heuvelink GBM, Ruiperez Gonzalez M, Kilibarda M, Blagotić A, et al. (2017) SoilGrids250m: Global gridded soil information based on machine learning. PLoS ONE 12(2): e0169748. doi:10.1371/journal.pone.0169748

In [30]:
# Copy RTI file from topo to soil directory
topo_rti_file = topo_dir + site_prefix + '.rti'
soil_rti_file = soil_dir + site_prefix + '.rti'
shutil.copyfile( topo_rti_file, soil_rti_file )

pedotransfer.save_soil_hydraulic_vars( site_prefix=site_prefix,
         in_dir=src_soil_dir, out_dir=soil_dir,
         out_bounds=out_bounds, REPORT=False,
         out_xres_sec=out_xres_sec, out_yres_sec=out_yres_sec,
         RESAMPLE_ALGO='bilinear')


Finished transforming ISRIC soil grid files.
   Number of grids = 35
   Number outside of model domain = 0

WARNING in read_soil_grid_files:
   Some values in OC grid are out of range.
   min(OC) = -9999.0
   Possible nodata value.

WARNING in read_soil_grid_files:
   Some values in D grid are out of range.
   min(D) = -9999.0
   Possible nodata value.

WARNING in wosten_theta_s:
   Some values are not in [0, 1].
   min(theta_s) = -9999.0
   Possible nodata value.
   Forcing bad values into range.

WARNING in wosten_alpha:
   Some values are out of typical range.
   Typical range: -0.15 to -0.004 [1/cm]
   max(alpha) = -0.00015224768 (excl. zero)
   max(alpha) = -0.0 (incl. zero)
   Forcing bad values into range.

ERROR in wosten_n:
   Some values are out of range.
   Typical range: 1.0 to 3.0 [unitless].
   Forcing bad values into range.

WARNING in get_tBC_from_vG_vars:
   Some values in G grid are out of range.
   Typical range: 0.08 to 2.3 [m].
   min(G) = 0.002772573 [m]
   max(G

## Create a Space-time Rainfall Rate Grid Stack  <a id="rainrates_B"></a>

<b>Note:</b> This capability has been delegated to the MINT data transformation team, since
downloading all of the required files can take hours to days.

However, assuming all required files have already been downloaded to your computer it can also be performed using TopoFlow's <b>regrid</b> utility.  The specific function for this is called:
<b>create_rts_from_nc_files()</b>.

For the [<b>MINT Project</b>](http://mint-project.info/), we have used two different
<b>global</b>, remote-sensing based products for rainfall from NASA, called:
<b>GPM</b> and <b>GLDAS</b>.

[<b>Version 06 of the GPM GPM IMERG Final Precipitation L3 </b>](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGHH_06/summary?keywords=GPM)
has a spatial resolution of <b>0.1 x 0.1 degrees</b> (360 x 360 arcseconds)
and a temporal resolution of <b>30 minutes</b> (1800 seconds). 
This data is available for dates between: <b>2000-06-01</b> and <b>2020-01-31</b>.
This is the highest spatial and temporal resolution available for GPM data.

[<b>Version 2.1 of the GLDAS Noah Land Surface Model L4</b>](https://disc.gsfc.nasa.gov/datasets/GLDAS_NOAH025_3H_2.1/summary?keywords=GLDAS)
has a spatial resolution of <b>0.25 x 0.25 degrees</b> (900 x 900 arcseconds)
and a temporal resolution of <b>3 hours</b> (180 minutes).
This data is available for dates between: <b>2000-01-01</b> and <b>2020-02-29</b>.
This is the highest spatial and temporal resolution available for GLDAS data.

In addition to space-time rainfall rates, some of the TopoFlow components can make use of other meteorological variables such as:  air temperature, soil temperature, relative humidity, surface wind speed, shortware radiation and longwave radiation.  The procedure for preparing these other space-time datasets for use in TopoFlow is very similar to what is done for precipitation.


In [5]:
# Try to open one of the GPM files
rain_dir = '/Users/peckhams/Data/GPM_Rain_Globe_2014/'
nc_rain_directory = rain_dir + '2014-08'
os.chdir( nc_rain_directory )
# var_name = 'precipitationCal'  ## Not present
var_name = 'HQprecipitation'
nc_file = '3B-HHR-E.MS.MRG.3IMERG.20140801-S000000-E002959.0000.V06B.HDF5.nc4'
(ds_in, grid, nodata) = regrid.gdal_open_nc_file( nc_file, var_name, VERBOSE=True)
ds_in = None  # (close the file)

Opening netCDF file...
Reading grid of values...
grid: min = -9999.9 , max = 81.54
grid.shape = (3600, 1800)
grid.dtype = float32
grid nodata = -9999.900390625
nodata count = 3783818



In [7]:
# Try to open one of the GPM files in a different set
nc_rain_directory = '/Users/peckhams/Data/GPM_Rain_2014-08_Ethiopia/'
os.chdir( nc_rain_directory )
var_name = 'precipitationCal'
nc_file = '3B-HHR.MS.MRG.3IMERG.20140801-S000000-E002959.0000.V06B.HDF5.nc4'
(ds_in, grid, nodata) = regrid.gdal_open_nc_file( nc_file, var_name, VERBOSE=True)
ds_in = None  # (close the file)

Opening netCDF file...
Reading grid of values...
grid: min = 0.0 , max = 15.618312
grid.shape = (171, 141)
grid.dtype = float32
grid nodata = -9999.900390625
nodata count = 0



In [6]:
# Convert one GPM file from netCDF to GeoTIFF
out_file = 'GPM_Conversion_Test.tif'
regrid.fix_gpm_file_as_geotiff( nc_file, var_name, out_file, 
                                out_nodata=None, VERBOSE=True)

Reading data from netCDF file...
grid: min   = -9999.9 , max = 81.54
grid.shape  = (3600, 1800)
grid.dtype  = float32
grid nodata = -9999.900390625
nodata count = 3783818
 
Rotating grid (90 deg ccw)...
Adjusting metadata...
geotransform  = (-89.99999694654582, 0.09999999660727314, 0.0, 179.99999694739424, 0.0, -0.09999999830410791)
geotransform2 = (-179.99999694739424, 0.09999999830410791, 0.0, 89.99999694654585, 0.0, -0.09999999660727314)
Writing grid to GeoTIFF file...
Finished.



In [8]:
# basin_dir   = '/Users/peckhams/TF_Tests3/Baro-Gam_60sec/'
# topo_dir    = basin_dir + '__topo/'
# met_dir     = basin_dir + '__met/'
from topoflow.utils import rti_files
site_prefix = 'Baro-Gam_60sec'
rti_file    = topo_dir + site_prefix + '.rti'
grid_info   = rti_files.read_info( rti_file )

from topoflow.utils import regrid

## GPM = False    # (use this for GLDAS data)
GPM = True  # (use this for GPM data)
NC4 = True

if (GPM):
    ## rain_dir = '/Users/peckhams/Data/GPM_Rain_Globe_2014/'
    ## nc_rain_directory = rain_dir + '2014-08'
    nc_rain_directory = '/Users/peckhams/Data/GPM_Rain_2014-08_Ethiopia/'
    # rts_file = met_dir + 'GPM_Rain_2014-08_Bilinear.rts'
    rts_file = met_dir + 'GPM_Rain_2014-08_Nearest.rts'
else:
    rain_dir = '/Users/peckhams/Data/GLDAS_Rain_2014-08_Ethiopia/'
    nc_rain_directory = rain_dir
    rts_file = met_dir + 'GLDAS_Rain_2014-08_TEST.rts'

os.chdir( nc_rain_directory )
regrid.create_rts_from_nc_files( rts_file=rts_file, NC4=NC4,
           GPM=GPM, VERBOSE=False, DEM_bounds=out_bounds,
           DEM_xres_sec=out_xres_sec, DEM_yres_sec=out_yres_sec,
           DEM_ncols=grid_info.ncols, DEM_nrows=grid_info.nrows,
           ## resample_algo='bilinear')
           resample_algo='nearest')


Working...
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 1
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 2
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 3
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 4
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 5
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 6
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 7
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 8
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 9
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 10
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 11
resaving as GeoTIFF...
comparing bounds...
regridding to DEM grid...
count = 12
resaving as GeoTIFF...
comparing bound

In [10]:
import imageio

ModuleNotFoundError: No module named 'visvis'

### Experiment 1:  Rainfall RTS File for Lol-Kuru, South Sudan


<b>Note:</b> This is a markdown cell with code, for reference.

``` python
#--------------------------------------------
# DEM information for Lol-Kuru, South Sudan
#--------------------------------------------
DEM_xres_sec = 30.0
DEM_yres_sec = 30.0
DEM_ncols    = 483
DEM_nrows    = 364
minlat = 6.532916666667    # (south)
maxlat = 9.566250000000    # (north)
minlon = 23.995416666666   # (west)
maxlon = 28.020416666666   # (east)
DEM_bounds2 = [minlon, minlat, maxlon, maxlat]

# Note: These GPM rainfall files apparently contain some NaN values.
rts_file = met_dir + 'GPM_Rain_2014-08-Lol-Kuru.rts'
nc_rain_directory = met_dir + 'GPM_Files_2014-08_South_Sudan/'
NC4 = False
GPM = True
resample_algo = 'bilinear'
## resample_algo = 'nearest'

os.chdir( nc_rain_directory )
regrid.create_rts_from_nc_files( rts_file=rts_file, NC4=NC4,
           GPM=GPM, VERBOSE=False, DEM_bounds=DEM_bounds2,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows,
           resample_algo=resample_algo)
```

### Experiment 2: Rainfall RTS File for Pongo, South Sudan

<b>Note:</b> This is a markdown cell with code, for reference.

``` python
#--------------------------------------------
# DEM information for "Pongo", South Sudan
#--------------------------------------------
DEM_xres_sec = 30.0
DEM_yres_sec = 30.0
DEM_ncols    = 396
DEM_nrows    = 428
DEM_bounds2 = [24.079583333333,  6.565416666666, 27.379583333333, 10.132083333333]
## DEM_bounds = [6.565416666666, 24.079583333333,  10.132083333333, 27.379583333333]

rts_file = met_dir + 'GPM_Rain_2014-08-Pongo.rts'
nc_rain_directory = met_dir + 'GPM_Files_2014-08_South_Sudan/'
NC4 = False
GPM = True
resample_algo = 'bilinear'
## resample_algo = 'nearest'

os.chdir( nc_rain_directory )
regrid.create_rts_from_nc_files( rts_file=rts_file, NC4=NC4,
           GPM=GPM, VERBOSE=False, DEM_bounds=DEM_bounds2,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows,
           resample_algo=resample_algo)
```

## Appendix 1: &nbsp; Installing TopoFlow in a conda Environment  <a id="setup_B"></a>

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to install the required Python packages in a conda environment called <b>tf36</b>.  This prevents conflicts with other Python packages you may have installed.
The Anaconda distribution includes many packages from the
[<b>Python Standard Library</b>](https://docs.python.org/3/library/).

First, download the TopoFlow 3.6 package from GitHub repo "topoflow36" at:
<b>https://github.com/peckhams/topoflow36</b>.
Copy or unzip the package into some directory on your computer.  Let's refer to this full path as TF36_DIR.  e.g. TF36_DIR = /Users/peckhams/Dropbox/TopoFlow_3.6

Installing TopoFlow 3.6 with pip causes most of its dependencies to be installed automatically.  However, if you want to run the growing collection of Jupyter notebooks (such as this one) that highlight TopoFlow functionality, you will also need to install the
[<b>nb_conda</b>](https://docs.anaconda.com/anaconda/user-guide/tasks/use-jupyter-notebook-extensions/) package, and optionally the <b>jupyterlab</b> package (see below).
In addition, some of the new TopoFlow utilities (e.g. regrid.py and visualize.py) require the
[<b>gdal</b>](https://pypi.org/project/GDAL/) and
[<b>matplotlib</b>](https://matplotlib.org/) packages.
Simply type the following commands at an OS prompt after installing Anaconda and downloading TopoFlow.

``` bash
% conda update -n base conda
% conda create --name tf36
% conda activate tf36
% conda list
% conda install nb_conda
% conda install gdal
% conda install matplotlib
% conda install imageio
% pip install imageio-ffmpeg  (an imageio extension)
% cd TF36_DIR
% pip install -e .   (-e is the editable/developer option)
```

<b>Note:</b>  The <b>pip</b> package manager is used to install TopoFlow 3.6, since it is not available as a conda package.  (It gets installed when you install <b>nb_conda</b>.) However, like conda, pip will install a package and its dependencies into the currently active conda environment, as explained
[<b>in these docs</b>](https://docs.conda.io/projects/conda/en/4.6.1/user-guide/tasks/manage-pkgs.html#installing-non-conda-packages).  If you switch to another environment with <b>conda activate envname</b>, you can confirm that topoflow (or cfunits, gdal, netcdf4, etc.) is not there with <b>conda list</b>. 

<b>Note:</b>  Some of the new TopoFlow utilities use a Python package version of [<b>GDAL</b>](https://pypi.org/project/GDAL/).  At the time of this writing, installing <b>gdal</b> from the <b>conda-forge </b> with <i>conda install -c conda-forge gdal</i> did not work.

<b>Note:</b> The netCDF4 package will be installed as a TopoFlow dependency.

#### <b>Conda Environments</b>

Note that <b>conda</b> is the name of the package manager for the popular Anaconda Python distribution.  One feature of conda is support for multiple environments, which are isolated from one another.  When you install Anaconda, an environment called <b>base</b> is created for you and a base set of commonly-used Python packages are installed there.  However, you can (and should!) create additional, named environments and install different sets of Python packages into them without worrying about potential conflicts with packages in other environments.  Type <b>conda env list</b> to list your available environments.  You can switch to one of your other environments using the command <b>conda activate envname</b>.  (Replace "envname" with the name of an environment.) You can switch back to the base environment with the command <b>conda deactivate</b>.  It is better not to install new packages into the base environment.  See the online conda documentation on [<b>Managing Environments</b>](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html) for more information.

It is always a good idea to update conda itself before creating new environments and installing packages in them. The "-n" flag is followed by the name of the environment to update, and the "-c" flag is followed by the name of the <b>channel</b> from which to get packages.  A channel is a collection of Python packages that are provided and maintained by some group.  The word "defaults" refers to
[<b>Anaconda's own collection</b>](https://docs.anaconda.com/anaconda/packages/pkg-docs/), while
[<b>conda-forge</b>](https://conda-forge.org/feedstocks/)
refers to another popular collection and the GitHub organization that maintains it.  Many Python packages are available from both of these channels.  (However, the ipyleaflet and pydap  packages are currently not available in the Anaconda collection.) When you are installing several packages into an environment, the potential for installation problems seems to be less if you get them all from the same channel.  Keep in mind that packages you install will likely depend on many other Python packages, so there is a potential for conflicts, usually related to different package versions.  Using conda environments helps to mitigate against this and helps with <b>reproducibility</b>.

Once you've switched to an environment with <b>conda activate envname</b>, you can type <b>conda list</b> to see a list of packages.  If you do this right after you create a new environment you will see that it contains no packages.  If you do this right after installing each package above you will see that:

<ul>
    <li>Installing <b>nb_conda</b> triggers installation of <b>nb_conda_kernels</b> (2.2.3),
        <b>ipykernel</b> (5.3.0), <b>notebook</b> (6.0.3), <b>pip</b> (20.0.2),
        <b>setuptools</b> (46.4.0) and <b>traitlets</b> (4.3.3), among many others.
    <li>Installing <b>gdal</b> triggers installation of #######.
    <li>Installing <b>matplotlib</b> triggers installation of ##########. 
</ul>

#### <b>Jupyter Notebook Extensions</b>

Note that <b>nb_conda</b> is installed first above, and triggers installation of <b>nb_conda_kernels</b> along with <b>notebook</b>.  This is important as it makes your Jupyter notebook app aware of your conda environments and available in the app as "kernels".  Anaconda provides a helpful page on the
[<b>Jupyter Notebook Extensions</b>](https://docs.continuum.io/anaconda/user-guide/tasks/use-jupyter-notebook-extensions/).
That page also explains how you can enable or disable these extensions individually. The command <b>jupyter nbextension list</b> shows you the extensions that are installed and whether they are enabled.  If you run the <b>jupyter notebook</b> or <b>jupyter lab</b> command in an environment that has <b>nb_conda_kernels</b> installed (see below), you will have the ability to associate one of your available conda environments with any new notebook you create.  Different environments give rise to different <b>kernels</b> in Jupyter, and the kernel name includes the environment name, e.g. <b>Python \[conda env:tf36\]</b>.  The kernel name is displayed in the upper right corner.  Notebooks typically open with the "environment kernel" they were created with. However, there is a <b>Change Kernel</b> option in the <b>Kernel</b> menu in the Jupyter app menu bar. (After changing the kernel, you may need to choose <b>Restart</b> from the <b>Kernel</b> menu.

#### <b>Cloning a conda Environment</b>

If your notebook is working but then you want to import additional packages (possibly with many dependencies, and potential for problems), you can keep the first environment but clone it with
<b><i>conda create --name clonename --copy --clone envname</i></b>,
and then install the additional packages in the clone.  This way, you can switch to the new environment's kernel and try to run your notebook, but if you run into any problems you can easily revert back to the original environment and functionality.

<b>Note:</b> Setting the "--copy" flag installs all packages using copies instead of hard or soft links.  This is necessary to avoid problems when using <b>pip</b> together with <b>conda</b> as described [<b>on this page</b>](https://stackoverflow.com/questions/43879119/installing-tensorflow-in-cloned-conda-environment-breaks-conda-environment-it-wa).

#### <b>Running Notebooks in the Jupyter Notebook App</b>

When you want to run the notebook, type <b>conda activate tf36</b> (at an OS command prompt) to activate this environment.  Then change to the directory that contains this notebook and type <b>jupyter notebook</b>.  By default, this folder is called <b>Jupyter</b> and is in your home directory.  In the app, choose this notebook by name, "TopoFlow_Getting_Started.ipynb", and make sure to choose the kernel called:  <b>Python \[conda env:tf36\]</b>.  See the References section at the end for more info.

#### <b>Running Notebooks in the JupyterLab App</b>

The
[<b>JupyterLab</b>](https://jupyterlab.readthedocs.io/en/stable/index.html)
app is a cool, new successor to the Notebook app and offers many additional features.  If you want to use this notebook in JupyterLab, you need to install one more Python package, as follows.

``` bash
% conda activate tf36
% conda install -c conda-forge jupyterlab
```

You launch the JupyterLab app by typing <b>jupyter lab</b> instead of <b>jupyter notebook</b>.  To quit, choose <b>Logout</b> or <b>Shutdown</b> from the app's <b>File</b> menu.

#### <b>JupyterLab Extensions</b>

The Jupyter project provides documentation on
[<b>JupyterLab Extensions</b>](https://jupyterlab.readthedocs.io/en/stable/user/extensions.html)
which add capabilities to JupyterLab.  For example, after installing jupyterlab (see just above), if you want to use the <b>ipywidgets</b> and <b>ipyleaflet</b> Python packages, you need to install two extensions, as follows:
```
% conda activate tf36
% jupyter labextension install jupyter-leaflet
% jupyter labextension install @jupyter-widgets/jupyterlab-manager
```
To list the jupyter labextensions you have, and to see whether or not they are enabled, type <b>jupyter labextension list</b>.  <b>Note:</b> If you start jupyterlab from a conda environment in which a given extension is not installed, and then open or switch to a notebook which uses a different "environment kernel", one that requires that extension, the notebook may not work.

You should only install trusted extensions, due to security concerns, as explained in the documentation.  Third-party extensions pose a potential security risk.  An extension that allows you to play MP4 movie files in JupyterLab is
[<b>available on GitHub</b>](https://github.com/jupyterlab/jupyterlab-mp4)
(by Ian Rose of the JupyterLab organization)
and can be installed with the command:
```
% jupyter labextension install @jupyterlab/mp4-extension
```
<b>Note:</b> This command is from a pull-request on the extension's github repo.
Using the command: "jupyter labextension install jupyterlab-mp4" results in the
following error message:<br>
An error occured.
ValueError: "jupyterlab-mp4" is not a valid npm package

As of May 27, 2020, JupyterLab has added an experimental <b>Extension Manager</b> which can be enabled by choosing Settings > Enable Extension Manager in the app.
